In [33]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in dataset 1
data = pd.read_excel("dataset_1.xls")

# Data Cleaning

From the initial viewing of the data, we noticed that many instances had missing data for some dates (usually tdate10 through tdate12). For now, we have set those values to NaN.

We also noticed whitespace occurred in the columns. We erased them and lowercased the names for simplicity.

In [3]:
# Example of an instance without data for each date
data.iloc[0,:]

State                          11
 Herd                           1
# PTAs                        581
 PTAM                         272
PTAF                           17
PTAP                           11
RHA # cows                    590
 % W PTAs                      98
 RHAM                       29810
 RHAF                        1093
 RHAP                         921
Tdate1        2018-08-15 00:00:00
 Milk1                         86
 Fat1                         3.4
 Pro1                         3.1
Tdate2        2018-07-21 00:00:00
 Milk2                         87
 Fat2                         3.5
 Pro2                           3
Tdate3        2018-06-13 00:00:00
 Milk3                         86
 Fat3                         3.4
 Pro3                           3
Tdate4        2018-05-16 00:00:00
 Milk4                         86
 Fat4                         3.4
 Pro4                         3.1
Tdate5        2018-04-11 00:00:00
 Milk5                         80
 Fat5         

In [4]:
# Remove whitespace from column names
data.columns = list(map(lambda x: x.strip().lower(), data.columns))

# Find values of '  .' and set to NaN
for col in data.columns:
    try:
        data[col][data[col] == '  .'] = np.nan
        data[col][data[col] == ' . '] = np.nan
        data[col][data[col].apply(lambda x: x == '    .')] = \
            (data[col][data[col].apply(lambda x: x != '    .')]).mean()
        if "date" not in col:
            data[col] = data[col].astype(float)
    except:
        continue
        
data["rhap"][data.rhap.apply(lambda x: x == '   .')] = data.rhap[data.rhap.apply(lambda x: x != '   .')].mean()
data["rhaf"][data.rhaf.apply(lambda x: x == '   .')] = data.rhaf[data.rhaf.apply(lambda x: x != '   .')].mean()

# Data Analysis

In [5]:
# Dimensions of data
data.shape

(7919, 59)

In [6]:
# Initial summary statistics
data.describe()

,state,herd,# ptas,ptam,ptaf,ptap,rha # cows,% w ptas,rham,milk1,...,pro9,milk10,fat10,pro10,milk11,fat11,pro11,milk12,fat12,pro12
count,7919.000000,7919.000000,7919.000000,7919.000000,7919.000000,7919.000000,7919.000000,7919.000000,7919.000000,7916.000000,...,6383.000000,5882.000000,5866.000000,5866.000000,5101.000000,5091.000000,5091.000000,3806.000000,3798.000000,3798.000000
mean,31.738603,3960.000000,142.206971,150.691375,9.027024,5.764996,216.199899,61.779265,22959.591208,61.218924,...,3.150807,62.027882,3.875230,3.140811,61.725740,3.805637,3.096700,62.246978,3.779568,3.064007
std,11.987906,2286.162724,355.044773,356.828884,12.999496,10.967446,441.442926,28.500609,4041.978663,12.303426,...,0.134838,11.673558,0.320722,0.137292,11.787126,0.314055,0.127777,11.525574,0.301237,0.122081
min,11.000000,1.000000,1.000000,-2940.000000,-108.000000,-99.000000,2.000000,0.000000,462.000000,2.000000,...,2.500000,15.000000,2.200000,2.500000,17.000000,2.400000,2.500000,15.000000,2.400000,2.500000
25%,23.000000,1980.500000,31.000000,-2.500000,3.000000,1.000000,60.000000,44.000000,20590.500000,54.000000,...,3.100000,55.000000,3.700000,3.100000,54.000000,3.600000,3.000000,55.000000,3.600000,3.000000
50%,31.000000,3960.000000,57.000000,213.000000,11.000000,7.000000,95.000000,71.000000,23321.000000,62.000000,...,3.100000,63.000000,3.900000,3.100000,62.000000,3.800000,3.100000,63.000000,3.800000,3.100000
75%,41.000000,5939.500000,117.000000,370.000000,17.000000,12.000000,188.000000,84.000000,25659.500000,70.000000,...,3.200000,70.000000,4.100000,3.200000,70.000000,4.000000,3.200000,70.000000,3.900000,3.100000
max,74.000000,7919.000000,9789.000000,1234.000000,63.000000,44.000000,9008.000000,250.000000,36275.000000,129.000000,...,3.700000,100.000000,6.600000,3.800000,102.000000,6.200000,3.700000,100.000000,5.900000,3.700000


In [7]:
# Amount of unique states the herds are from
len(data.state.unique())

39

In [8]:
# Amount of unique herds, since each row is a unique herd, should be the same amount
len(data.herd.unique())

7919

# Data Visualizations

# Model Creation

In [56]:
milk_columns = []
for col in data.columns:
    if "milk" in col:
        milk_columns.append(col)
feature_cols = ['# ptas', 'ptam', 'ptaf', 'ptap', 'rha # cows',
                '% w ptas', 'rham', 'rhaf', 'rhap']

X = data.loc[:, feature_cols]
y = data.loc[:, milk_columns].mean(axis=1, skipna=True)

X = X[~y.isna()]
y = y[~y.isna()]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lasso = Lasso(alpha=5)

cross_val_score(lasso, X_train, y_train, cv=5)

lasso.fit(X_train, y_train)
lasso.score(X_test, y_test)

0.9848636173130209